In [9]:
%matplotlib inline
import os
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

df_train=pd.read_csv('../data/train.csv', parse_dates=['date'])

In [10]:
df_train_clean=df_train.loc[df_train['label']!=-1]
del df_train
y=df_train_clean['label']
X=df_train_clean.drop(['id','label','date'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=69)

In [11]:
num_values=1000
category_feature=[]
for i in range(X.shape[1]):
    num = X['f'+str(i+1)].unique().shape[0]
    if num <=num_values:
        category_feature.append(i)

In [12]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [22]:
lgb_train=lgb.Dataset(x_train,y_train,free_raw_data=False)
lgb_valid=lgb.Dataset(x_test,y_test,reference=lgb_train,free_raw_data=False)
params = {'boosting_type': 'gbdt',
          'objective': 'binary', 
          'metric': 'auc', 
          'device': 'cpu', 
          'verbose': 0, 
          'learning_rate': 0.01, 
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'feature_fraction': 0.9,
          'lambda_l1': 0.006, 
          'num_leaves': 128,
          'early_stopping_round': 20
}
feature_name=['feasture_'+str(col+1) for col in range(x_train.shape[1])]

In [23]:
gbm=lgb.train(params, lgb_train, num_boost_round=10000, valid_sets=lgb_valid, valid_names='valid_set', feval=atec_metric, verbose_eval=500)

[500]	valid_set's auc: 0.989652	valid_set's atec_metric: 0.638842
[1000]	valid_set's auc: 0.991434	valid_set's atec_metric: 0.666354
[1500]	valid_set's auc: 0.991802	valid_set's atec_metric: 0.675043
[2000]	valid_set's auc: 0.992012	valid_set's atec_metric: 0.683475
[2500]	valid_set's auc: 0.992133	valid_set's atec_metric: 0.689693
[3000]	valid_set's auc: 0.992224	valid_set's atec_metric: 0.692589
[3500]	valid_set's auc: 0.992284	valid_set's atec_metric: 0.693441
[4000]	valid_set's auc: 0.992391	valid_set's atec_metric: 0.696082
[4500]	valid_set's auc: 0.992385	valid_set's atec_metric: 0.697189
[5000]	valid_set's auc: 0.99242	valid_set's atec_metric: 0.698126
[5500]	valid_set's auc: 0.992452	valid_set's atec_metric: 0.700256
[6000]	valid_set's auc: 0.992494	valid_set's atec_metric: 0.700426
[6500]	valid_set's auc: 0.992454	valid_set's atec_metric: 0.7023
[7000]	valid_set's auc: 0.99247	valid_set's atec_metric: 0.703322
[7500]	valid_set's auc: 0.992486	valid_set's atec_metric: 0.704429


In [25]:
df_test=pd.read_csv('../data/test.csv', parse_dates=['date'])
X_test=df_test.drop(['id','date'],axis=1)

In [27]:
y_test = gbm.predict(X_test)

In [32]:
np.savetxt(os.getcwd()+"/prediction/lgb_10000_1.txt", y_test, delimiter=',')